# Mounting Gdrive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Libraries

In [ ]:
cd 'your code directory'

In [6]:
import keras
import pickle
import numpy as np
from tqdm import tqdm
from skimage import data
from scipy import ndimage
from skimage import filters
import matplotlib.pyplot as plt
import pickle
import numpy as np
from tqdm import tqdm
from skimage import data
from skimage import filters
from skimage import data
from skimage import filters
from scipy import ndimage
from skimage import data
from skimage import filters
import matplotlib.pyplot as plt
import pickle


def Plot_Slice(my_data,my_out,npreds,mask_scaled,LungMask,NumCase,NumSlice):

    plt.figure(figsize=(15,15))
    plt.subplot(161) # original
    plt.imshow(my_data[NumCase][0,NumSlice,:,:,0])

    plt.subplot(162) # removed
    plt.imshow(my_out[NumCase][NumSlice,:,:,0])

    plt.subplot(163) # generated mask
    plt.imshow(npreds)

    plt.subplot(164) # Real mask
    plt.imshow(mask_scaled[NumCase][0,NumSlice,:,:,0])

def sensitivityScore(label_trues, label_preds):
    sensitivities = []
    for i in range(label_trues.shape[0]):
        true_positives = (label_trues[i] * label_preds[i]).sum()
        possible_positives = label_trues[i].sum()
        score = (true_positives / (possible_positives + 1e-9)) if possible_positives > 0 else 0
        sensitivities.append(score)
    return np.max(sensitivities) if sensitivities else 0

def specificityScore(label_trues, label_preds):
    specificities = []
    for i in range(label_trues.shape[0]):
        true_negatives = ((1 - label_trues[i]) * (1 - label_preds[i])).sum()
        possible_negatives = (1 - label_trues[i]).sum()
        score = (true_negatives / (possible_negatives + 1e-9)) if possible_negatives > 0 else 0
        specificities.append(score)
    return np.max(specificities) if specificities else 0

def DiceScore(label_trues, label_preds):
    dice_scores = []
    for i in range(label_trues.shape[0]):
        intersection = 2 * (label_trues[i] * label_preds[i]).sum()
        sum_labels = label_trues[i].sum() + label_preds[i].sum()
        score = (intersection / (sum_labels + 1e-9)) if sum_labels > 0 else 0
        dice_scores.append(score)
    return np.max(dice_scores) if dice_scores else 0


def PlotingResults(dice_list,InfectionPercent,ScoreName=''):
    dice_list = np.array(dice_list)
    InfectionPercent = np.array(InfectionPercent)
    Data_up1per= dice_list[np.where(( (InfectionPercent> 1) ))]
    print(ScoreName,"Mean:", 100*np.array(Data_up1per)[~np.isnan(Data_up1per)].mean() ,ScoreName,"Std:", 100*np.array(Data_up1per)[~np.isnan(Data_up1per)].std() )


def EvaluateMain(my_data,my_out,mask_scaled,LungMask,Threshold,thresholdMethod,PlotingCase,PlotingSlice,PostPro):
    maxval=1
    dice_list_p = []
    sensitivity_list_p = []
    specificity_list_p = []
    InfectionPercent = []

    for i in tqdm(range(len(my_data))):
        for j in range(my_data[i].shape[1]):
            InfectionPercent.append( (100*mask_scaled[i][0,j,:,:,0].sum())/(LungMask[i][j,:,:].sum()+(10**-9)) )
            diff = my_out[i][j,:,:,0]



            npreds = (diff > Threshold*np.amax(diff)) * maxval



            dice_list_p.append(DiceScore(np.reshape(mask_scaled[i][0,j,:,:,0],(128*128)), np.reshape( npreds ,(128*128) )))


            sensitivity_list_p.append(sensitivityScore(np.reshape(mask_scaled[i][0,j,:,:,0],(128*128)), np.reshape( npreds ,(128*128) )))


            specificity_list_p.append(specificityScore(np.reshape(mask_scaled[i][0,j,:,:,0],(128*128)), np.reshape( npreds ,(128*128) )))

    return dice_list_p ,sensitivity_list_p ,specificity_list_p ,InfectionPercent

# Load The Model

In [3]:
### load the model
BCDU_net_D3_model = keras.models.load_model('/content/drive/MyDrive/my_en_de')
BCDU_net_D3_model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 128, 128, 1)]        0         []                            
                                                                                                  
 conv2d_56 (Conv2D)          (None, 128, 128, 32)         320       ['input_4[0][0]']             
                                                                                                  
 conv2d_57 (Conv2D)          (None, 128, 128, 32)         9248      ['conv2d_56[0][0]']           
                                                                                                  
 max_pooling2d_12 (MaxPooli  (None, 64, 64, 32)           0         ['conv2d_57[0][0]']           
 ng2D)                                                                                      

# Load Data

In [4]:
#### calculate the output of model
def OutPutModel(my_data):
    my_out = []
    for i in tqdm(range(len(my_data))):
        my_out.append(BCDU_net_D3_model.predict(my_data[i][0,:,:,:]))
    return my_out


def Add_Dim(data):
    ################## add a dim to data
    data_scaled = []
    for j in tqdm(range(len(data))):
            output = np.zeros((1,data[j].shape[0],data[j].shape[1],data[j].shape[2],1))
            output[0,:,:,:,0] = data[j][:,:,:]
            data_scaled.append(output)
    return data_scaled


def LoadData(DatasetName):

    ############## Load
    if(DatasetName=='Mosmed_Dataset'):
        Base_data = np.load('/content/drive/MyDrive/Groupe1/preprocessed_data/BaseCTImages.npy')

        data = np.load('/content/drive/MyDrive/Groupe1/preprocessed_data/LungImages.npy')


        mask = np.load('/content/drive/MyDrive/Groupe1/preprocessed_data2/InfectionMasks.npy')

        LungMask = np.load('/content/drive/MyDrive/Groupe1/preprocessed_data2/LungMask.npy')

        for i in range(mask.shape[0]):
            mask[i,:,:,:]=np.where(mask[i,:,:,:]>0.5,1,0)
            LungMask[i,:,:,:]=np.where(LungMask[i,:,:,:]>0.0005,1,0)


    if(DatasetName=='UESTC'):
        Base_data = np.load('./BaseCTImages.npy')

        data = np.load('./LungImages.npy')

        mask = np.load('./InfectionMasks.npy')

        LungMask = np.load('./LungMask.npy')
        for i in range(len(mask)):
            mask[i]=np.where(np.array(mask[i])>0.5,1,0)
            LungMask[i]=np.where(np.array(LungMask[i])>0.0005,1,0)


    if(DatasetName=='Radiopedia'):
        Base_data = np.load('./BaseCTImages.npy')

        data = np.load('./LungImages.npy')

        mask = np.load('./InfectionMasks.npy')

        LungMask = np.load('./LungMask.npy')
        for i in range(len(mask[0].shape[0])):
            mask[i]=np.where(np.array(mask[i])>0.5,1,0)
            LungMask[i]=np.where(np.array(LungMask[i])>0.0005,1,0)



    ####################### Add a dim
    ########### add a dim to data
    Data_scaled = Add_Dim(data)

    ########### add a dim to mask
    mask_scaled = Add_Dim(mask)

    ### get output of model
    my_out = OutPutModel(Data_scaled)

    return Data_scaled,my_out, mask_scaled, LungMask





In [8]:
def EvaluateMain(my_data, my_out, mask_scaled, LungMask, Threshold, thresholdMethod, PlotingCase, PlotingSlice, PostPro):
    maxval = 1
    dice_list_p = []
    sensitivity_list_p = []
    specificity_list_p = []
    InfectionPercent = []

    for i in tqdm(range(len(my_data))):
        for j in range(my_data[i].shape[1]):
            # Ensure that i and j are within valid range
            if i < len(mask_scaled) and j < my_data[i].shape[1]:
                # Check dimensions of mask_scaled and LungMask
                if mask_scaled[i][0, j, :, :, 0].shape == LungMask[i][j, :, :].shape:
                    infection_sum = mask_scaled[i][0, j, :, :, 0].sum()
                    lung_mask_sum = LungMask[i][j, :, :].sum()

                    # Check if lung mask sum is zero and skip calculations
                    if lung_mask_sum > 0:
                        infection_percent = (100 * infection_sum) / (lung_mask_sum + (10**-9))
                        InfectionPercent.append(infection_percent)

                        diff = my_out[i][j, :, :, 0]

                        npreds = (diff > Threshold * np.amax(diff)) * maxval

                        dice_list_p.append(DiceScore(np.reshape(mask_scaled[i][0, j, :, :, 0], (128*128)),
                                                     np.reshape(npreds, (128*128))))

                        sensitivity_list_p.append(sensitivityScore(np.reshape(mask_scaled[i][0, j, :, :, 0], (128*128)),
                                                                   np.reshape(npreds, (128*128))))

                        specificity_list_p.append(specificityScore(np.reshape(mask_scaled[i][0, j, :, :, 0], (128*128)),
                                                                   np.reshape(npreds, (128*128))))
                    else:
                        print(f"Warning: Lung mask sum is zero for case {i}, slice {j}. Skipping calculations.")
                else:
                    print("Error: Dimensions of mask_scaled and LungMask do not match.")


    return dice_list_p, sensitivity_list_p, specificity_list_p, InfectionPercent


# Evaluation

In [9]:
#### upload data
my_data, my_out, mask_scaled, LungMask = LoadData("Mosmed_Dataset")

####### Calculating the evaluation metrics(Dice score, Specificity, Sensitivity )

dice_list, sensitivity_list, specificity_list, InfectionPercent = EvaluateMain(my_data,my_out,mask_scaled,LungMask,Threshold= 0.2,thresholdMethod='1',
                                                                                      PlotingCase=10,PlotingSlice=15,PostPro=False)


  0%|          | 0/50 [00:00<?, ?it/s]

1/1 [==============================] - 0s 52ms/step


  2%|▏         | 1/50 [00:00<00:18,  2.65it/s]

1/1 [==============================] - 0s 39ms/step


  4%|▍         | 2/50 [00:00<00:13,  3.58it/s]

...

 98%|█████████▊| 49/50 [00:09<00:00,  5.06it/s]

1/1 [==============================] - 0s 33ms/step


  0%|          | 0/50 [00:00<?, ?it/s]

  2%|▏         | 1/50 [00:10<08:13, 10.06s/it]

100%|██████████| 50/50 [00:18<00:00,  2.65it/s]

# Plot The Results

In [10]:
############### ploting resuts

### Dice score
PlotingResults(dice_list ,InfectionPercent,ScoreName='Dice score')

##### Sensitivity
PlotingResults(sensitivity_list,InfectionPercent,ScoreName='Sensitivity')


##### Specificity
PlotingResults(specificity_list,InfectionPercent,ScoreName='specificity')



Dice score Mean: 72.72727269090907 Dice score Std: 44.536177119244236
Sensitivity Mean: 72.72727265454544 Sensitivity Std: 44.536177096976154
specificity Mean: 99.99999989999996 specificity Std: 3.3306690738754696e-14


In [11]:
!pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
aiohttp                          3.9.1
aiosignal                        1.3.1
alabaster                        0.7.13
albumentations                   1.3.1
altair                           4.2.2
anyio                            3.7.1
appdirs                          1.4.4
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array-record                     0.5.0
arviz                            0.15.1
astropy                          5.3.4
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.0
attrs                            23.1.0
audioread                        3.0.1
autograd                         1.6.2
Babel                            2.14.0
backcall                         0.2.0
beautifulsoup4                   4.11.2
bidict                           0.22.1
b